# Trabajo Final

###  CLASIFICACION CON Feed-Forward Networks - Perceptrón Multicapa (MLP)

### Universidad de Antioquia

### Facultad de Ingeniería

### Ingeniería de Sistemas

### Ude@ - 2018-I

#### Profesor: Antonio Tamayo Herrera

### Integrantes:

Nombre: David de Jesus Yepes Isaza

Cédula: 1020455542

Nombre: Rodrigo Antonio Porras Martinez

Cédula: 1039459720

### Clasificación en el dataset "Heart Disease Data Set" con Feed-Forward Networks - Perceptrón Multicapa (MLP)

La base de datos con las que se trabaja en el presente artículo fue extraída de  Cleveland Clinic Foundation y fue creada por V.A. Medical Center, Long Beach and Cleveland Clinic Foundation: Robert Detrano, M.D., Ph.D.

El set de datos de de Cleveland consta de 303 instancias, la variable de salida o etiqueta son 0 para la ausencia y los valores 1, 2, 3 y 4 para la presencia de enfermedades. 
 
El 54% de las instancias tienen ausencia de enfermedades y 46% tienen presencia de enfermedades del corazón. Para simplificar el problema se analiza como un problema biclase donde  “0” son personas sanas y “1” que corresponde a (1,2,3,4) son personas enfermas.

El objetivo es según las características de cada paciente identificar cuál de ellos presenta presencia de enfermedades y cuáles no.

La base de datos contiene 303 muestras con las 14 características siguientes:
1. N° 3 (edad)   
2. N° 4 (sexo) 
3. N° 9 (cp) tipo de dolor en el pecho
        Valor 1: angina típica
        Valor 2: angina atípica
        Valor 3: dolor no anginal
        Valor 4: asintomático  
4. N° 10 (trestbps) presión arterial en reposo
5. N° 12 (chol) colesterol sérico en mg / dl (miligramo por decilitro)
6. N° 16 (fbs) (azúcar en la sangre en ayunas > 120 mg / dl) (1 = verdadero, 0 = falso)
7. N° 19 (restecg)  resultados electrocardiográficos en reposo:
        Valor 0: normal
        Valor 1: tiene anormalidad de onda ST-T (inversiones de onda T y / o ST  elevación o depresión de > 0.05 mV)
        Valor 2: muestra hipertrofia ventricular izquierda probable o definitiva según los criterios de Estes  
8. N° 32 (thalach)  frecuencia cardíaca máxima lograda
9. N° 38 (exang) angina inducida por el ejercicio (1 = sí; 0 = no)
10. N° 40 (oldpeak) depresión del ST inducida por el ejercicio en relación con el reposo
11. N° 41 (pendiente) la pendiente del segmento ST de ejercicio máximo
        Valor 1: pendiente ascendente
        Valor 2: plano
        Valor 3: downsloping
12. N° 44 (ca) número de vasos principales (0-3) coloreados por fluoroscopia
13. N° 51 (thal) 3 = normal; 6 = defecto fijo; 7 = defecto reversible
14. N° 58 (num) (el atributo predicho) diagnóstico de enfermedad cardíaca (estado angiográfico de la enfermedad)
        - Valor 0: <50% de diámetro de estrechamiento
        - Valor 1:> 50% de diámetro de estrechamiento

### Importacion de Librerias y creacion de funciones

In [1]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
import neurolab as nl
from sklearn.model_selection import KFold
import time
from sklearn.metrics import confusion_matrix
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.preprocessing import normalize
from sklearn.neural_network import MLPClassifier as MLP

def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

def config_net_parameters(X):
    config = []
    for i in range(0,np.size(X,1)):
        maxXi = int(max(X[:,i]))
        minXi = int(min(X[:,i]))
        config.append([minXi,maxXi])
    
    #config = config[:-1] + "]"
    
    return config


### Eliminacion de valores Perdidos (Missing Values)

En la base de datos algunos valores están perdidos, se pueden identificar por el valor “-9.0”. por lo que se deben limpiar.

In [2]:
#cargamos la bd de entrenamiento
db = np.loadtxt('cleveland.txt',delimiter=',')  # El delimitador es ","
print( "Dim de la base de datos original: " + str(np.shape(db)))
db = db.reshape(303,14)
DataBase = db
#La base d edatos processed.cleveland tiene valores perdidos, vamos a quitar todos esos registros de la base de datos
#Están marcados como -9.0. Donde haya un valor -9.0 eliminaremos el registro.
j = 0
for i in range(0,np.size(db,0)):
    if -9.0 == db[i,13]:
        j+=1
        DataBase = np.delete(DataBase,i,0)
    elif 1 <= db[i,13]: #Se cambia la variables de salida (1,2,3 y 4) por el valor 1 para obtener un problema biclase
        DataBase[i,13]=1
    
print ("\nHay " + str(j) + " valores perdidos en la variable de salida. :(")

print ("\nDim de la base de datos sin las muestras con variable de salida perdido "+ str(np.shape(DataBase)))

#Ya hemos eliminado los registros con valor de la variable de salida perdido.

#Ahora vamos a imputar los valores perdidos en cada una de las características
print ("\nProcesando imputación de valores perdidos en las características . . .\n")

for k in range(0,np.size(DataBase,0)):
    for w in range(0,14):
        if -9.0 == DataBase[k,w]:
            DataBase[k,w] = round(np.mean(DataBase[:,w]))
        
print ("Imputación finalizada.\n")

hay_missed_values = False
for i in range(0,np.size(DataBase,0)):
    if -9.0 in DataBase[i,:]:
        hay_missed_values = True
if(hay_missed_values):
    print ("Hay valores perdidos")
else:
    print ("No hay valores perdidos en la base de datos. Ahora se puede procesar. La base de datos está en la variable DataBase")

print("Dimension: "+ str(np.shape(DataBase)))
#print(DataBase)

X = DataBase[:,0:13]


print ("Lista de listas con los máximos y mínimos de cada característica. La va a necesitar para crear el modelo.\n")
print (config_net_parameters(X))

Y = DataBase[:,13]
Y = Y.reshape(np.size(X,0),1)
#W=np.squeeze(np.asarray(Y))
#print(np.shape(W))
#print(W)

Dim de la base de datos original: (303, 14)

Hay 0 valores perdidos en la variable de salida. :(

Dim de la base de datos sin las muestras con variable de salida perdido (303, 14)

Procesando imputación de valores perdidos en las características . . .

Imputación finalizada.

No hay valores perdidos en la base de datos. Ahora se puede procesar. La base de datos está en la variable DataBase
Dimension: (303, 14)
Lista de listas con los máximos y mínimos de cada característica. La va a necesitar para crear el modelo.

[[29, 77], [0, 1], [1, 4], [94, 200], [126, 564], [0, 1], [0, 2], [71, 202], [0, 1], [0, 6], [1, 3], [0, 3], [3, 7]]


In [3]:
#Para calcular el costo computacional
tiempo_i = time.time()

net = nl.net.newff(config_net_parameters(X),[20,1],[nl.trans.LogSig(),nl.trans.LogSig()])

#Implemetamos la metodología de validación cross validation con 10 folds

Errores = np.ones(10)
Sensibilidad = np.ones(10)
Especificidad = np.ones(10)
Precision = np.ones(10)
Eficiencia = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  

    
    # Entrenar el modelo
    err = net.train(X_train,y_train,epochs=100)

    # Validación del modelo
    ypred = net.sim(X_test)
    
    #Se llevan los valores a 1 o 0 para los que está definido el modelo 
    pos = 0
    for tag in ypred:
        
        if (tag > 0.5):
            ypred[pos] = 1
        elif (tag <= 0.5):
            ypred[pos] = 0
        
        pos += 1
    
    Errores[j] = classification_error(ypred, y_test)
    tn, fp, fn, tp = confusion_matrix(y_test, ypred).ravel()
    Sensibilidad[j] = tp/(tp+fn)
    Especificidad[j] = tn/(tn+fp)
    Precision[j] = tp/(tp+fp)
    Eficiencia[j] = (tp+tn)/(tp+tn+fp+fn)
    j+=1
    

print("\nSensibilidad: "+str(np.mean(Sensibilidad)))
print("\nEspecificidad: "+str(np.mean(Especificidad)))
print("\nPrecision: "+str(np.mean(Precision)))
print("\nEficiencia: "+str(np.mean(Eficiencia)))
print("\nError de validación: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))
print (("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i)) + " segundos.\n")

c:\users\highwind\appdata\local\programs\python\python36-32\lib\site-packages\neurolab\trans.py:107: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch: 100; Error: 5.2269929576569165;
The maximum number of train epochs is reached
Epoch: 100; Error: 5.002134692672357;
The maximum number of train epochs is reached
Epoch: 100; Error: 5.00184173550964;
The maximum number of train epochs is reached
Epoch: 100; Error: 5.500789996472602;
The maximum number of train epochs is reached
Epoch: 100; Error: 5.000709343498565;
The maximum number of train epochs is reached
Epoch: 100; Error: 4.500977823279264;
The maximum number of train epochs is reached
Epoch: 100; Error: 5.003748564964612;
The maximum number of train epochs is reached
Epoch: 100; Error: 5.500274379938535;
The maximum number of train epochs is reached
Epoch: 100; Error: 4.500156418286035;
The maximum number of train epochs is reached
Epoch: 100; Error: 5.000662213689608;
The maximum number of train epochs is reached

Sensibilidad: 0.8971587725999491

Especificidad: 0.9165079979360164

Precision: 0.8985026248261543

Eficiencia: 0.907741935483871

Error de validación: 0.09225

<table>
  <tr>
    <th># de Epocas</th>
    <th># Neuronas en capa oculta</th>
    <th>Sensibilidad</th>
    <th>Especificidad</th>
    <th>Precision</th>
    <th>Eficiencia</th>
    <th>Error de validación</th>
    <th>Tiempo de ejecución (s)</th>
  </tr>
  <tr>
    <td>10</td>
    <td>10</td>
    <td>0.7463295772854597</td>
    <td>0.8160808128919583</td>
    <td>0.7846986346986347</td>
    <td>0.7864516129032258</td>
    <td>0.21354838709677418 +/- 0.10526239175530988
</td>
    <td>3.875757932662964</td>
  </tr>
  <tr>
    <td>10</td>
    <td>20</td>
    <td>0.7966437484084543</td>
    <td>0.8699907857879993</td>
      <td>0.8462867937016544</td>
    <td>0.838494623655914</td>
      <td>0.16150537634408602 +/- 0.07989732108765989</td>
    <td>4.45756983757019</td>
  </tr>
  <tr>
    <td>10</td>
    <td>40</td>
    <td>0.6916539343009932</td>
    <td>0.7673124836979326</td>
      <td>0.7252809252151359</td>
    <td>0.730752688172043</td>
      <td>0.269247311827957 +/- 0.11853267152555388</td>
    <td>9.402490615844727</td>
  </tr>
  <tr>
    <td>100</td>
    <td>10</td>
    <td>0.9175859434682965</td>
    <td>0.9064910296102248</td>
      <td>0.8949732130614484</td>
    <td>0.9143010752688173</td>
      <td>0.08569892473118279 +/- 0.042243287246351124</td>
    <td>29.13865900039673</td>
  </tr>
  <tr>
    <td>100</td>
    <td>20</td>
    <td>0.879712248535778</td>
    <td>0.9515686274509804</td>
      <td>0.9416666666666668</td>
    <td>0.9210752688172044</td>
      <td>0.0789247311827957 +/- 0.04657838138245579</td>
    <td>56.3459370136261</td>
  </tr>
  <tr>
    <td>100</td>
    <td>40</td>
    <td>0.8703835625159154</td>
    <td>0.8993274475782217</td>
      <td>0.884636393018746</td>
    <td>0.8878494623655915</td>
      <td>0.1121505376344086 +/- 0.05780317531038013</td>
    <td>101.29404807090759</td>
  </tr><tr>
    <td>500</td>
    <td>10</td>
    <td>0.8691112808759869</td>
    <td>0.9254721362229101</td>
     <td>0.9191666666666667</td>
    <td>0.900967741935484</td>
      <td>0.09903225806451613 +/- 0.053637379090120556</td>
    <td>5.44825291633606</td>
  </tr>
  <tr>
    <td>500</td>
    <td>20</td>
    <td>0.9175859434682965</td>
    <td>0.9143767507002801</td>
      <td>0.9077731092436976</td>
    <td>0.917741935483871</td>
      <td>0.08225806451612902 +/- 0.044335121065355795</td>
    <td>13.267746210098267</td>
  </tr>
  <tr>
    <td>500</td>
    <td>40</td>
    <td>0.883511586452763</td>
    <td>0.9325438596491228</td>
      <td>0.9175879675879676</td>
    <td>0.9080645161290322</td>
      <td>0.09193548387096774 +/- 0.06470268725846613</td>
    <td>290.020037651062</td>
  </tr>
</table>


Cuando se aumenta las neuronas de la capa oculta aumenta el tiempo, esto es porque los son muchos mas pesos a calcular por cada posible entrada de las neuronas de salida. Por otra parte aumentar las neuronas no implica que se obtiene un error menor. Aumentar las neuronas aumenta el tiempo, pero aumentar las epocas lo hace mucho mas ya que implica las veces que se recorre la red neuronal, por eso el tiempo que aumenta tanto cuando se pasa de 10 epocas a 100 o 500. Con lo que concluimos que no necesariamente la cantidad de neuronas indica un mejor resultado, pero aplicar mas epocas si ayuda a disminuir el error pero con un mayor costo computacional.

## Selección de Caracteristicas - Wrappers

Para selecionar las caracteristicas usaremos la funcion Wrappers y como objetivo MLP. los parámetros son el número de características a seleccionar  (3, 7 y 10) y la estrategia a implementar (SFS, SBS, SFFS, SBFS)


In [4]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
scores =['accuracy', 'f1', 'precision', 'recall']
#Feature Selection Function
#Recibe 4 parámetros: 1. el modelo (net para nuestro caso), 2. el número de características final que se quiere alcanzar
#3. Si es forward (True), si es Backward False, 4. Si es es flotante (True), sino False
def select_features(modelo, n_features, fwd, fltg, score):
    
    sfs = SFS(modelo, 
           k_features=n_features, 
           forward=fwd,
           floating=fltg,
           verbose=1,
           scoring=score,
           cv=0)
    
    return sfs

#Configurando modelo MLP de SKLEARN
clf=MLP(hidden_layer_sizes=(20,),activation='logistic', solver='sgd',max_iter=100)

#Para calcular el costo computacional
tiempo_i = time.time()

#Implemetamos la metodología de validación cross validation con 10 folds

Medidas = np.ones(10)
MedidasPromedio=np.ones(4)
j = 0
k = 0
kf = KFold(n_splits=10)

for scr in scores:
    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        

        sf = select_features(clf,10,False,True,scr)


        sf = sf.fit(X_train,y_train)


        Medidas[j] = sf.k_score_
        j+=1
    
    MedidasPromedio[k]=np.mean(Medidas)
    k+=1
    j=0

print("\nError en validación : " + str((1-MedidasPromedio[0])*100) + "%" )
print("\nEficiencia : " + str(MedidasPromedio[0]*100) + "%" )
print("\nPresicion : " + str(MedidasPromedio[2]*100) + "%" )
print("\nSensibilidad : " + str(MedidasPromedio[3]*100) + "%" )
print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")


c:\users\highwind\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\highwind\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.5s finished
Features: 12/10[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 11/10[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 10

c:\users\highwind\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.3s finished
Features: 12/10[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 11/10[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 11/10[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 10/10c:\users\highwind\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\neural

[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.4s finished
Features: 12/10[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 11/10[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 10/10c:\users\highwind\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\highwind\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarnin

c:\users\highwind\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.5s finished
Features: 12/10[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 11/10[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 10/10c:\users\highwind\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\highwind\app

Features: 12/10[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 11/10[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 10/10c:\users\highwind\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\highwind\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
c:\users\highwind\appdata\local\programs\python\python36-32\lib\sit

[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.4s finished
Features: 12/10[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 11/10[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
Features: 10/10c:\users\highwind\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\highwind\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarnin


Error en validación : 29.81173238526179%

Eficiencia : 70.18826761473821%

Presicion : 81.98145844172704%

Sensibilidad : 68.4351675190139%


Tiempo total de ejecución: 65.69493961334229 segundos.


<table>
  <tr>
    <th>Técnica</th>
    <th># de Caracteristicas</th>
    <th>Error de validación</th>
    <th>Eficiencia</th>
     <th>Precision</th>
      <th>Sensibilidad</th>
    <th>Tiempo de ejecución (s)</th>
  </tr>
    <tr>
    <td>MLP + SFS</td>
    <td>3</td>
    <td>29.589797457444522%</td>
    <td>70.41020254255548%</td>
    <td>83.8668157321408%</td>
    <td>47.70162215758857%</td>
    <td>39.955190896987915</td>
  </tr>
  <tr>
    <td>MLP + SFS</td>
    <td>7</td>
     <td>28.162842059900882%</td>
    <td>71.83715794009912%</td>
    <td>91.44715865538598%</td>
    <td>51.73992860651438%</td>
    <td>94.86458826065063</td>
  </tr>
  <tr>
    <td>MLP + SFS</td>
    <td>10</td>
     <td>33.88224520577461%</td>
    <td>66.11775479422539%</td>
    <td>80.13147155413395%</td>
    <td>25.60102974939057%</td>
    <td>112.05507731437683</td>
  </tr>
  <tr>
    <td>MLP + SBS</td>
    <td>3</td>
     <td>31.09553436759318%</td>
    <td>68.90446563240683%</td>
    <td>58.78649271033892%</td>
    <td>56.236664565356655%</td>
    <td>126.2105507850647</td>
  </tr>
  <tr>
    <td>MLP + SBS</td>
    <td>7</td>
     <td>30.325226244343884%</td>
    <td>69.67477375565612%</td>
    <td>76.91278290576726%</td>
    <td>60.63310245525446%</td>
    <td>97.04789757728577</td>
  </tr>
  <tr>
    <td>MLP + SBS</td>
    <td>10</td>
     <td>30.289000215470807%</td>
    <td>69.71099978452919%</td>
    <td>82.28468214754636%</td>
    <td>64.44050108719954%</td>
    <td>57.32961916923523</td>
  </tr>
  <tr>
    <td>MLP + SFFS</td>
    <td>3</td>
     <td>34.28719564748977%</td>
    <td>65.71280435251023%</td>
    <td>88.14453671286519%</td>
    <td>54.5341314549449%</td>
    <td>45.29747915267944</td>
  </tr>
  <tr >
    <td>MLP + SFFS</td>
    <td>7</td>
     <td>30.984028226675285%</td>
    <td>69.01597177332471%</td>
    <td>81.54690979347906%</td>
    <td>57.73025842351232%</td>
    <td>141.5731213092804 </td>
  </tr>
  <tr>
    <td>MLP + SFFS</td>
    <td>10</td>
    <td>35.09978991596638%</td>
    <td>64.90021008403362%</td>
    <td>67.41888805236765%</td>
    <td>53.698571969061845%</td>
    <td>262.4194016456604</td>
  </tr>
  <tr>
    <td>MLP + SBFS</td>
    <td>3</td>
     <td>30.436193708252524%</td>
    <td>69.56380629174748%</td>
    <td>63.12281580596662%</td>
    <td>46.36564582231624%</td>
    <td>284.64475774765015</td>
  </tr>
  <tr>
    <td>MLP + SBFS</td>
    <td>7</td>
     <td>29.95892587804352%</td>
    <td>70.04107412195648%</td>
    <td>78.66021921341071%</td>
    <td>62.71053419761128%</td>
    <td>136.6192011833191</td>
  </tr>
  <tr >
    <td>MLP + SBFS</td>
    <td>10</td>
     <td>29.99461322990735%</td>
    <td>70.00538677009264%</td>
    <td>84.44341637364164%</td>
    <td>63.8029320099917%</td>
    <td>63.8029320099917%</td>
  </tr>    
</table>


El mejor de los resultados que obtuvimos fue de una eficiencia del 71% con 10 caracteristicas y la estrategia SBS. Ninguno de los resultados anteriores supero el  92% que obtuvo el Modelo MLP con todas las caracteristicas. Lo que nos lleva a pensar que Para este modelo la seleccion de caracteristicas no es optimo porque hace que se pierda mucha informacion perjudicando la eficiencia del modelo.

## Reducción de dimensión por extracción de características con PCA

In [17]:
from mlxtend.preprocessing import standardize
from mlxtend.feature_extraction import PrincipalComponentAnalysis as PCA


#Feature Extraction Function

def extract_features(tipo, n):
    if tipo == 'pca':
        ext = PCA(n_components=n)
        return ext
    elif tipo == 'lda':
        ext = LDA(n_discriminants=n)
        return ext
    else:
        print ("Ingrese un método válido (pca o lda)\n")

#Para calcular el costo computacional
tiempo_i = time.time()

#Estandarizamos los datos
X = standardize(X)

clf=MLP(hidden_layer_sizes=(20,),activation='logistic', solver='sgd',max_iter=100)
#Implemetamos la metodología de validación cross validation con 10 folds

Errores = np.ones(10)
Sensibilidad = np.ones(10)
Especificidad = np.ones(10)
Precision = np.ones(10)
Eficiencia = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
    
    ex = extract_features('pca',9) 

    #Fit de PCA
    ex = ex.fit(X)
    
    #Transforme las variables y genere el nuevo espacio de características de menor dimensión
    #complete el código aquí para hacer la transformación
    X_ex = ex.transform(X)
      
    #Se aplica CV-10
    
    X_train, X_test = X_ex[train_index], X_ex[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  
   
    #Aquí se entrena y se valida el modelo luego de aplicar extracción de características con PCA o LDA
    
    ######
    
    
    # Entrenamiento el modelo.
    model = clf.fit(X_train,y_train)

    # Validación del modelo
    ypred = model.predict(X_test)
        
    #######

    Errores[j] = classification_error(ypred, y_test)
    tn, fp, fn, tp = confusion_matrix(y_test, ypred).ravel()
    Sensibilidad[j] = tp/(tp+fn)
    Especificidad[j] = tn/(tn+fp)
    Precision[j] = tp/(tp+fp)
    Eficiencia[j] = (tp+tn)/(tp+tn+fp+fn)
    j+=1
    

print("\nError de validación: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))
print("\nSensibilidad: "+str(np.mean(Sensibilidad)))
print("\nEspecificidad: "+str(np.mean(Especificidad)))
print("\nPrecision: "+str(np.mean(Precision)))
print("\nEficiencia: "+str(np.mean(Eficiencia)))
print (("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i)) + " segundos.\n")


c:\users\highwind\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)



Error de validación: 0.3602150537634409 +/- 0.1151050031614777

Sensibilidad: 0.2766507512095747

Especificidad: 0.96468782249742

Precision: nan

Eficiencia: 0.6397849462365591


Tiempo total de ejecución: 0.6128032207489014 segundos.



c:\users\highwind\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in longlong_scalars


<table>
  <tr>
    <th>Técnica</th>
    <th># de Comp. Principales</th>
    <th>Error de validación</th>
    <th>IC (std)</th>
      <th>Sensibilidad</th>
      <th>Especificidad</th>
      <th>Precision</th>
      <th>Eficiencia</th>
    <th>Tiempo de ejecución</th>
  </tr>
    <tr>
    <td>MLP + PCA</td>
    <td>2</td>
    <td>0.455</td>
    <td>0.0750</td>
        <td>0.0521</td>
    <td>0.9949</td>
        <td></td>
    <td>0.557</td>
    <td>0.58481</td>
  </tr>
  <tr>
    <td>MLP + PCA</td>
    <td>6</td>
    <td> 0.3690</td>
      <td>0.1335</td>
    <td>0.2787</td>
      <td>0.940</td>
    <td>0.73658</td>
    <td>0.6309</td>
    <td>0.5678</td>
  </tr>
  <tr>
    <td>MLP + PCA</td>
    <td>10</td>
    <td>0.3902</td>
      <td>0.13440</td>
    <td>0.226</td>
      <td>0.9538</td>
    <td></td>
    <td>0.6097</td>
    <td>0.6138</td>
  </tr>
  <tr>
    <td>MLP + PCA</td>
    <td>12</td>
    <td>0.3396</td>
      <td>0.1069</td>
    <td>0.29419</td>
      <td>0.98421</td>
    <td>0.9666</td>
    <td>0.6603</td>
    <td>0.5658</td>
  </tr>
      
</table>
